In [1]:
from utils.sampling import *
from utils.config import cfg_from_yaml_file
from tools import builder
from datasets.data_transforms import ResamplePoints
import open3d as o3d
import numpy as np
import glob
from utils.vis_utils import convert_to_o3dpcd

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [5]:
import torch 

cfg_path = '/VCN/cfgs/VCN_models/VCN_VC.yaml'
cfg = cfg_from_yaml_file(cfg_path)
model = builder.model_builder(cfg.model)
model.cuda()

ckpt_path = 'model_zoo/VCN_VC.pth'
state_dict = torch.load(ckpt_path, map_location=f'cuda:0')
base_ckpt = {k.replace("module.", ""): v for k, v in state_dict['base_model'].items()}
model.load_state_dict(base_ckpt)

model.eval()

VCN_VC(
  (pose_encoder): Sequential(
    (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (3): LeakyReLU(negative_slope=0.01)
    (4): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
    (5): AdaptiveMaxPool1d(output_size=1)
  )
  (pose_fc): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): Linear(in_features=512, out_features=9, bias=True)
  )
  (encoder): FeatureEncoder(
    (mlp_conv1): Sequential(
      (0): Conv1d(3, 128, kernel_size=(1,), stride=(1,))
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
    )
    (mlp_conv2): Sequential(
      (0): Conv1d(512, 512, kernel_size=(1,), stride=(1,))
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [6]:
# Read data
pcd_dir = 'data/demo'
pcd_data = glob.glob(pcd_dir + '/*.pcd')

idx = 4
in_pcd = o3d.io.read_point_cloud(pcd_data[idx])

In [7]:
# Complete car

resample = ResamplePoints({'n_points': 1024})

in_dict = {}
in_dict['input'] = torch.from_numpy(resample(np.asarray(in_pcd.points))).float().cuda().unsqueeze(0)
ret_dict = model(in_dict)
pred_surface = partial_with_KDTree( in_dict['input'][0], ret_dict['coarse'][0], k=20)
output = get_largest_cluster(pred_surface.detach().cpu().numpy(), eps=0.3, min_points=1)

In [8]:
# Visualize
vis_list = []

vis_list.append(convert_to_o3dpcd(in_dict['input'].squeeze(0).cpu().numpy(), [0.14, 0.34, 0.69]))
vis_list.append(convert_to_o3dpcd(ret_dict['coarse'].squeeze(0).detach().cpu().numpy(), [0.70, 0.85, 0.94]))
vis_list.append(convert_to_o3dpcd(output))

o3d.visualization.draw_geometries(vis_list)